In [7]:
import pandas as pd     
import os 

**Group MSA**

**Metropolitan Statistical Area 1964**

Clean the 99crosswalk.csv and save it as crosswalk.csv

In [ ]:
import pandas as pd     
import os              

Use crosswalk.csv to aggragate cbp_census_64 to MSA

In [ ]:
#Load files to work with
cbp_census_64 = pd.read_csv('../input/MIRROR_task1_output/cbp_census_64.csv')
crosswalk = pd.read_csv('../input/MIRROR_task4_output/crosswalk.csv')

cbp_census_64 = cbp_census_64.drop(columns = ['Unnamed: 0'])
cbp_census_64['fips'] = cbp_census_64['fips'].astype(str).str.zfill(5)
crosswalk = crosswalk.sort_values('fips', ascending = True)
crosswalk['fips'] = crosswalk['fips'].astype(str).str.zfill(5)
crosswalk['msa'] = crosswalk['msa'].astype(str).str.zfill(4)

# Merge the county employment data with the crosswalk on the 'fips' column
cbp_census_64 = cbp_census_64.merge(crosswalk, on='fips', how='left')
cbp_census_64 = cbp_census_64.dropna(subset=['msa'])
cbp_census_64 = cbp_census_64.drop_duplicates(subset=['fips', 'msa'])

# Print the count of duplicate 'fips' values
duplicates_count = cbp_census_64['fips'].duplicated().sum()
print(f"Number of duplicate 'fips' values: {duplicates_count}")
# Create a boolean mask for duplicated values in the 'fips' column
duplicates_mask = cbp_census_64['fips'].duplicated(keep=False)
# Use the mask to filter the DataFrame and see the duplicated values
## 10 counties are part of multiple MSA
duplicated_values = cbp_census_64[duplicates_mask]
duplicates = pd.DataFrame(duplicated_values)

#Create Wage
cbp_census_64['wage'] = cbp_census_64['ap'] / cbp_census_64['emp'] *1000

cbp_census_64

Number of duplicate 'fips' values: 10


,fips,year,ap,emp,total,male,female,white,non-white,drop,hs,some-coll,coll-more,mean-age,rent,msa,wage
0,01001,1964,13136.0,4072.0,18739,9021,9718,10839,7900,6249,3646,868,727,28.912802,57.983871,5240,3225.933202
1,01003,1964,41252.0,13838.0,49088,24514,24574,38759,10329,16892,8028,2150,1890,29.064048,71.596284,5160,2981.066628
4,01009,1964,12360.0,4081.0,25449,12718,12731,24613,836,10329,3365,561,395,30.832017,54.544776,1000,3028.669444
7,01015,1964,142880.0,37123.0,95878,46601,49277,77805,18073,28938,13356,3611,3712,28.252415,72.050349,0450,3848.826873
16,01033,1964,140740.0,25026.0,46506,22807,23699,37524,8982,14401,7808,2069,1505,28.931858,63.258993,2650,5623.751299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,55131,1964,116132.0,24349.0,46119,23326,22793,46060,59,13370,11453,2649,2350,29.436978,115.273077,5082,4769.477186
3676,55133,1964,313544.0,63781.0,158249,79774,78475,157958,291,34551,43377,14740,17162,28.357146,138.603774,5082,4915.946755
3679,55139,1964,369980.0,71112.0,107928,52642,55286,107667,261,25708,23740,5990,6464,31.168056,117.644400,0460,5202.778715
3691,56021,1964,95020.0,21804.0,60149,30909,29240,58716,1433,9482,11138,4031,3735,28.283978,87.736056,1580,4357.915979


Aggregate to MSA level

In [ ]:
# Group the data by 'msa' and keep 'year' as is
msa1964 = cbp_census_64.groupby('msa')['year'].max().reset_index()
# Calculate the sum of 'total' and 'emp' columns
msa1964['total'] = cbp_census_64.groupby('msa')['total'].sum().reset_index()['total']
msa1964['emp'] = cbp_census_64.groupby('msa')['emp'].sum().reset_index()['emp']

# Calculate the employment-weighted average for other columns
columns_to_average = ['drop', 'hs', 'some-coll', 'coll-more', 'rent', 'male', 'female', 'white', 'non-white', 'mean-age', 'wage']
for column in columns_to_average:
    weighted_avg = (cbp_census_64[column] * cbp_census_64['emp']).groupby(cbp_census_64['msa']).sum() / cbp_census_64.groupby(cbp_census_64['msa'])['emp'].sum()
    msa1964[column] = weighted_avg.reset_index()[0]
# Reset the index (optional)
msa1964 = msa1964.reset_index(drop=True)

Create weighted average and Combine 1964 and 1965 into a single year (1964).

Find average age (average of median)

Convert Race, Gender, Education to ratio

In [ ]:
#Gender Ratio
msa1964['Male'] = msa1964['male'] / (msa1964['male'] + msa1964['female'])
msa1964['Female'] = msa1964['female'] / (msa1964['male'] + msa1964['female'])
msa1964['Male'] = msa1964['Male'].round(4)
msa1964['Female'] = msa1964['Female'].round(4)

#Race Ratio
msa1964['White'] = msa1964['white'] / (msa1964['white'] + msa1964['non-white'])
msa1964['Non-white'] = msa1964['non-white'] / (msa1964['white'] + msa1964['non-white'])
msa1964['White'] = msa1964['White'].round(4)
msa1964['Non-white'] = msa1964['Non-white'].round(4)

#Education Ratio
msa1964['Drop'] = msa1964['drop'] / msa1964[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa1964['Hs'] = msa1964['hs'] / msa1964[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa1964['Some-coll'] = msa1964['some-coll'] / msa1964[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa1964['Coll-more'] = msa1964['coll-more'] / msa1964[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa1964 = msa1964[['msa', 'year', 'wage', 'total', 'emp', 'Drop', 'Hs', 'Some-coll', 'Coll-more', 'rent', 'Male', 'Female', 'White', 'Non-white', 'mean-age']]
msa1964.to_csv('../output/msa1964.csv')

**Metropolitan Statistical Area 2009**

Use crosswalk.csv to aggragate cbp_acs_09 to MSA

In [8]:
#Load files to work with
cbp_acs_09 = pd.read_csv('../input/MIRROR_task1_output/cbp_acs_09.csv')
crosswalk = pd.read_csv('../input/MIRROR_task4_output/crosswalk.csv')

cbp_acs_09 = cbp_acs_09.drop(columns = ['Unnamed: 0'])
cbp_acs_09['fips'] = cbp_acs_09['fips'].astype(str).str.zfill(5)
crosswalk = crosswalk.sort_values('fips', ascending = True)
crosswalk['fips'] = crosswalk['fips'].astype(str).str.zfill(5)
crosswalk['msa'] = crosswalk['msa'].astype(str).str.zfill(4)

# Merge the county employment data with the crosswalk on the 'fips' column
cbp_acs_09 = cbp_acs_09.merge(crosswalk, on='fips', how='left')
cbp_acs_09 = cbp_acs_09.dropna(subset=['msa'])
cbp_acs_09 = cbp_acs_09.drop_duplicates(subset=['fips', 'msa'])

# Print the count of duplicate 'fips' values
duplicates_count = cbp_acs_09['fips'].duplicated().sum()
print(f"Number of duplicate 'fips' values: {duplicates_count}")
# Create a boolean mask for duplicated values in the 'fips' column
duplicates_mask = cbp_acs_09['fips'].duplicated(keep=False)
# Use the mask to filter the DataFrame and see the duplicated values
## 10 counties are part of multiple MSA
duplicated_values = cbp_acs_09[duplicates_mask]
duplicates = pd.DataFrame(duplicated_values)

#Create Wage
cbp_acs_09['wage'] = cbp_acs_09['ap'] / cbp_acs_09['emp'] *1000

cbp_acs_09

Number of duplicate 'fips' values: 10


,fips,year,ap,emp,total,male,female,white,non-white,drop,hs,some-coll,coll-more,mean-age,rent,msa,wage
0,01001,2009,551538,21418,53944,26174,27770,42577,12202,4685,12113,7688,10095,35.899868,832,5240,25751.143898
1,01003,2009,3073344,108357,179523,87553,91970,155068,26908,15094,36271,29344,43741,39.779415,863,5160,28363.132977
4,01009,2009,418205,14529,57140,28357,28783,54446,3430,10330,13839,7468,6836,38.152765,561,1000,28784.155826
7,01015,2009,2375312,79752,117614,56539,61075,88450,31284,17296,26319,17051,17440,38.081287,593,0450,29783.729562
16,01033,2009,1236550,39812,54478,26201,28277,44084,11151,7310,12819,8584,9279,40.235719,592,2650,31059.730734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715,55131,2009,3516648,99522,131190,65018,66172,126409,6100,6879,29099,20600,33331,38.433345,788,5082,35335.383131
3716,55133,2009,19752324,446036,388125,190403,197722,364113,29523,12788,68901,56613,128580,39.225248,894,5082,44284.147468
3719,55139,2009,7101361,168953,166033,83335,82698,154734,13961,10863,39140,23410,37014,37.849280,635,0460,42031.576829
3731,56021,2009,2139996,63936,90394,45020,45374,78828,15135,4608,16490,16529,21595,36.960086,698,1580,33470.908408


Aggregate to MSA level

In [9]:
# Group the data by 'msa' and keep 'year' as is
msa2009 = cbp_acs_09.groupby('msa')['year'].max().reset_index()

# Calculate the sum of 'total' and 'emp' columns
msa2009['total'] = cbp_acs_09.groupby('msa')['total'].sum().reset_index()['total']
msa2009['emp'] = cbp_acs_09.groupby('msa')['emp'].sum().reset_index()['emp']

# Calculate the employment-weighted average for other columns
columns_to_average = ['drop', 'hs', 'some-coll', 'coll-more', 'rent', 'male', 'female', 'white', 'non-white', 'mean-age', 'wage']
for column in columns_to_average:
    weighted_avg = (cbp_acs_09[column] * cbp_acs_09['emp']).groupby(cbp_acs_09['msa']).sum() / cbp_acs_09.groupby(cbp_acs_09['msa'])['emp'].sum()
    msa2009[column] = weighted_avg.reset_index()[0]

# Reset the index (optional)
msa2009 = msa2009.reset_index(drop=True)


In [11]:
#Gender Ratio
msa2009['Male'] = msa2009['male'] / (msa2009['male'] + msa2009['female'])
msa2009['Female'] = msa2009['female'] / (msa2009['male'] + msa2009['female'])
msa2009['Male'] = msa2009['Male'].round(4)
msa2009['Female'] = msa2009['Female'].round(4)

#Race Ratio
msa2009['White'] = msa2009['white'] / (msa2009['white'] + msa2009['non-white'])
msa2009['Non-white'] = msa2009['non-white'] / (msa2009['white'] + msa2009['non-white'])
msa2009['White'] = msa2009['White'].round(4)
msa2009['Non-white'] = msa2009['Non-white'].round(4)

#Education Ratio
msa2009['Drop'] = msa2009['drop'] / msa2009[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2009['Hs'] = msa2009['hs'] / msa2009[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2009['Some-coll'] = msa2009['some-coll'] / msa2009[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2009['Coll-more'] = msa2009['coll-more'] / msa2009[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)

msa2009 = msa2009[['msa', 'year', 'wage', 'total', 'emp', 'Drop', 'Hs', 'Some-coll', 'Coll-more', 'rent', 'Male', 'Female', 'White', 'Non-white', 'mean-age']]
msa2009
msa2009.to_csv('../output/msa2009.csv')
msa2009

,msa,year,wage,total,emp,Drop,Hs,Some-coll,Coll-more,rent,Male,Female,White,Non-white,mean-age
0,0040,2009,29304.201504,130463,104891,0.147788,0.287116,0.255676,0.309420,730.000000,0.4860,0.5140,0.8129,0.1871,35.752957
1,0120,2009,30555.280130,122867,89155,0.199073,0.298256,0.255731,0.246940,666.722360,0.4632,0.5368,0.3157,0.6843,35.358281
2,0160,2009,39279.277753,918216,696382,0.089454,0.277713,0.171528,0.461305,854.575170,0.4855,0.5145,0.8049,0.1951,38.733936
3,0200,2009,36050.104377,859226,590647,0.133056,0.242866,0.237767,0.386312,757.277431,0.4897,0.5103,0.6735,0.3265,36.669033
4,0220,2009,31241.093831,131254,98443,0.182315,0.350169,0.225924,0.241592,687.000000,0.4821,0.5179,0.6327,0.3673,37.036582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,2009,33534.905779,239836,127572,0.286445,0.270134,0.216588,0.226833,693.000000,0.5001,0.4999,0.7038,0.2962,34.247319
272,9280,2009,36538.681594,431967,323345,0.125457,0.418879,0.161250,0.294414,782.000000,0.4935,0.5065,0.8737,0.1263,38.549658
273,9320,2009,31400.863840,559732,374838,0.126144,0.428870,0.197854,0.247133,597.042963,0.4859,0.5141,0.8383,0.1617,40.731398
274,9340,2009,32662.890306,166009,61170,0.217639,0.249040,0.263833,0.269488,854.479941,0.4995,0.5005,0.6303,0.3697,34.947237


**Metropolitan Statistical Area 2019**

In [ ]:
#Load files to work with
cbp_acs_19 = pd.read_csv('../input/MIRROR_task1_output/cbp_acs_19.csv')
crosswalk = pd.read_csv('../input/MIRROR_task4_output/crosswalk.csv')
#rent23 = pd.read_csv('../input/MIRROR_task1_output/rent23.csv')
cbp_acs_19 = cbp_acs_19.drop(columns = ['Unnamed: 0'])
cbp_acs_19['fips'] = cbp_acs_19['fips'].astype(str).str.zfill(5)
#rent23['fips'] = rent23['fips'].astype(str).str.zfill(5)
crosswalk = crosswalk.sort_values('fips', ascending = True)
crosswalk['fips'] = crosswalk['fips'].astype(str).str.zfill(5)
crosswalk['msa'] = crosswalk['msa'].astype(str).str.zfill(4)

# Merge the county employment data with the crosswalk on the 'fips' column
cbp_acs_19 = cbp_acs_19.merge(crosswalk, on='fips', how='inner')
cbp_acs_19 =cbp_acs_19.dropna(subset=['msa'])
cbp_acs_19 = cbp_acs_19.drop_duplicates(subset=['fips', 'msa'])

# Print the count of duplicate 'fips' values
duplicates_count = cbp_acs_19['fips'].duplicated().sum()
print(f"Number of duplicate 'fips' values: {duplicates_count}")
# Create a boolean mask for duplicated values in the 'fips' column
duplicates_mask = cbp_acs_19['fips'].duplicated(keep=False)
# Use the mask to filter the DataFrame and see the duplicated values
## 10 counties are part of multiple MSA
duplicated_values = cbp_acs_19[duplicates_mask]
duplicates = pd.DataFrame(duplicated_values)

#Create Wage
cbp_acs_19['wage'] = cbp_acs_19['ap'] / cbp_acs_19['emp'] *1000
cbp_acs_19['year'] = cbp_acs_19['year'].astype(int)

cbp_acs_19


Number of duplicate 'fips' values: 10


,fips,year,ap,emp,total,male,female,white,non-white,drop,hs,some-coll,coll-more,mean-age,rent,msa,wage
0,01001,2019,759620,22907,55639,27052,28587,42150,14910,4273,11880,7663,14044,38.658818,1011.0,5240,33161.042476
1,01003,2019,4585339,131438,218289,105889,112400,186504,37462,14823,42272,34475,63993,41.703840,1032.0,5160,34885.946226
2,01009,2019,455926,13498,57755,28677,29078,54271,4777,6838,13975,8505,10496,40.076037,666.0,1000,33777.300341
3,01015,2019,2614775,73093,114324,54925,59399,83801,33375,11787,27203,19007,20970,39.818599,720.0,0450,35773.261461
4,01033,2019,1915562,47844,54957,26448,28509,43279,12810,5590,14452,9017,10018,41.383281,712.0,2650,40037.664075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,55131,2019,4861598,105967,135529,67250,68279,127569,10684,4688,28286,19772,43107,41.251559,925.0,5082,45878.414978
1458,55133,2019,27272739,491467,402637,198162,204475,365265,46943,10792,62070,57487,154400,41.445286,1078.0,5082,55492.513231
1459,55139,2019,9362111,177883,170924,86174,84750,155159,20110,7653,38053,23564,46334,39.278211,784.0,0460,52630.723565
1460,56021,2019,3084732,68849,99272,50481,48791,87433,17803,3956,17238,17265,29131,38.592297,933.0,1580,44804.310883


In [ ]:

# Group the data by 'msa' and keep 'year' as is
msa2019 = cbp_acs_19.groupby('msa', as_index=False)['year'].max()

# Calculate the sum of 'total' and 'emp' columns
total_emp_sum = cbp_acs_19.groupby('msa')[['total', 'emp']].sum().reset_index()
msa2019 = msa2019.merge(total_emp_sum, on='msa')

# Calculate the employment-weighted average for other columns
columns_to_average = ['drop', 'hs', 'some-coll', 'coll-more', 'rent', 'male', 'female', 'white', 'non-white', 'mean-age', 'wage']
for column in columns_to_average:
    weighted_avg = (cbp_acs_19[column] * cbp_acs_19['emp']).groupby(cbp_acs_19['msa']).sum() / cbp_acs_19.groupby(cbp_acs_19['msa'])['emp'].sum()
    msa2019[column] = weighted_avg.reset_index()[0]

msa2019


,msa,year,total,emp,drop,hs,some-coll,coll-more,rent,male,female,white,non-white,mean-age,wage
0,0040,2019,137521,114042,8848.000000,25733.000000,19869.000000,29786.000000,921.000000,67038.000000,70483.000000,103411.000000,40499.000000,35.905636,38691.262868
1,0120,2019,118431,81887,8158.471601,15619.425232,12721.369241,16774.273755,788.498785,38241.866548,44433.972486,22679.603734,61709.548903,37.459526,39549.513354
2,0160,2019,930060,754002,12231.080079,38204.533182,25899.613708,85877.684580,1037.673968,115366.199916,121644.510137,186526.525369,61347.500379,40.127015,51364.691340
3,0200,2019,900509,614254,42651.233710,98010.311142,98536.393269,186779.994895,908.495668,301719.864162,313881.732153,432288.725555,240476.216928,39.143978,43946.300716
4,0220,2019,130376,91409,11683.000000,31478.000000,19489.000000,24032.000000,825.000000,63029.000000,67347.000000,82162.000000,51893.000000,38.330383,40839.971994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,2019,250649,144429,39123.000000,41418.000000,31932.000000,39861.000000,868.000000,125145.000000,125504.000000,180076.000000,89652.000000,35.200280,42715.112616
272,9280,2019,447628,337350,30253.000000,122486.000000,51458.000000,108462.000000,972.000000,221169.000000,226459.000000,390226.000000,73912.000000,40.279206,44837.711576
273,9320,2019,530110,344472,13721.228518,57260.900880,29108.402274,42379.573222,678.429736,97337.880382,101361.157252,164925.754352,41006.293687,42.330374,38168.179126
274,9340,2019,173839,65792,11451.504864,13612.757873,15877.839859,17641.627098,1069.550097,45308.643756,45028.492218,59933.659624,39298.908104,36.745534,45086.834266


In [ ]:
#Gender Ratio
msa2019['Male'] = msa2019['male'] / (msa2019['male'] + msa2019['female'])
msa2019['Female'] = msa2019['female'] / (msa2019['male'] + msa2019['female'])
msa2019['Male'] = msa2019['Male'].round(4)
msa2019['Female'] = msa2019['Female'].round(4)

#Race Ratio
msa2019['White'] = msa2019['white'] / (msa2019['white'] + msa2019['non-white'])
msa2019['Non-white'] = msa2019['non-white'] / (msa2019['white'] + msa2019['non-white'])
msa2019['White'] = msa2019['White'].round(4)
msa2019['Non-white'] = msa2019['Non-white'].round(4)

#Education Ratio
msa2019['Drop'] = msa2019['drop'] / msa2019[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2019['Hs'] = msa2019['hs'] / msa2019[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2019['Some-coll'] = msa2019['some-coll'] / msa2019[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)
msa2019['Coll-more'] = msa2019['coll-more'] / msa2019[['drop', 'hs', 'some-coll', 'coll-more']].sum(axis=1)

msa2019 = msa2019[['msa', 'year', 'wage', 'total', 'emp', 'Drop', 'Hs', 'Some-coll', 'Coll-more', 'rent', 'Male', 'Female', 'White', 'Non-white', 'mean-age']]
msa2019.to_csv('../output/msa2019.csv')
msa2019

,msa,year,wage,total,emp,Drop,Hs,Some-coll,Coll-more,rent,Male,Female,White,Non-white,mean-age
0,0040,2019,38691.262868,137521,114042,0.105038,0.305487,0.235873,0.353602,921.000000,0.4875,0.5125,0.7186,0.2814,35.905636
1,0120,2019,39549.513354,118431,81887,0.153143,0.293193,0.238793,0.314871,788.498785,0.4626,0.5374,0.2688,0.7312,37.459526
2,0160,2019,51364.691340,930060,754002,0.075401,0.235521,0.159664,0.529413,1037.673968,0.4868,0.5132,0.7525,0.2475,40.127015
3,0200,2019,43946.300716,900509,614254,0.100125,0.230083,0.231318,0.438473,908.495668,0.4901,0.5099,0.6426,0.3574,39.143978
4,0220,2019,40839.971994,130376,91409,0.134780,0.363143,0.224833,0.277243,825.000000,0.4834,0.5166,0.6129,0.3871,38.330383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,2019,42715.112616,250649,144429,0.256824,0.271889,0.209618,0.261668,868.000000,0.4993,0.5007,0.6676,0.3324,35.200280
272,9280,2019,44837.711576,447628,337350,0.096760,0.391756,0.164582,0.346902,972.000000,0.4941,0.5059,0.8408,0.1592,40.279206
273,9320,2019,38168.179126,530110,344472,0.096310,0.401915,0.204312,0.297463,678.429736,0.4899,0.5101,0.8009,0.1991,42.330374
274,9340,2019,45086.834266,173839,65792,0.195472,0.232364,0.271028,0.301135,1069.550097,0.5016,0.4984,0.6040,0.3960,36.745534


Append 64, 09, 19 datasets

In [ ]:
import numpy as np
import pandas as pd

msa2019 = pd.read_csv('../input/MIRROR_task4_output/msa2019.csv')
msa2009 = pd.read_csv('../input/MIRROR_task4_output/msa2009.csv')
msa1964 = pd.read_csv('../input/MIRROR_task4_output/msa1964.csv')

msa64_19 = pd.concat([msa1964, msa2009, msa2019], ignore_index=True)
msa64_19 = msa64_19.sort_values(by=['msa', 'year'], ascending=[True, True])
msa64_19 = msa64_19.drop(columns = 'Unnamed: 0')
msa64_19 = msa64_19.rename(columns = {'Drop' : 'drop', 'Hs' : 'hs', 'Some-coll':'somecoll', 'Coll-more': 'collmore',
                                 'Male':'male', 'Female':'female', 'White':'white', 'Non-white':'nonwhite', 'mean-age':'age'})

msa64_19['logwage'] = np.log(msa64_19['wage'])
msa64_19['logrent'] = np.log(msa64_19['rent'])
msa64_19['logemp'] = np.log(msa64_19['emp'])

print(msa64_19.columns.to_list)

msa64_19 = msa64_19[['msa', 'year', 'wage', 'logwage', 'emp', 'logemp', 'rent', 'logrent' , 'drop', 'hs', 'somecoll',
       'collmore', 'male', 'female', 'white', 'nonwhite', 'age', 'total']]

msa64_19

<bound method IndexOpsMixin.tolist of Index(['msa', 'year', 'wage', 'total', 'emp', 'drop', 'hs', 'somecoll',
       'collmore', 'rent', 'male', 'female', 'white', 'nonwhite', 'age',
       'logwage', 'logrent', 'logemp'],
      dtype='object')>


,msa,year,wage,logwage,emp,logemp,rent,logrent,drop,hs,somecoll,collmore,male,female,white,nonwhite,age,total
0,40,1964,4060.505393,8.309063,45153.0,10.717812,85.011783,4.442790,0.432599,0.313833,0.131013,0.122554,0.5009,0.4991,0.9530,0.0470,28.023180,101078
274,40,2009,29304.201504,10.285486,104891.0,11.560677,730.000000,6.593045,0.147788,0.287116,0.255676,0.309420,0.4860,0.5140,0.8129,0.1871,35.752957,130463
550,40,2019,38691.262868,10.563369,114042.0,11.644322,921.000000,6.825460,0.105038,0.305487,0.235873,0.353602,0.4875,0.5125,0.7186,0.2814,35.905636,137521
1,120,1964,3713.002915,8.219596,34300.0,10.442901,80.534773,4.388689,0.512755,0.282281,0.110210,0.094755,0.4918,0.5082,0.6560,0.3440,25.610463,81884
275,120,2009,30555.280130,10.327293,89155.0,11.398132,666.722360,6.502374,0.199073,0.298256,0.255731,0.246940,0.4632,0.5368,0.3157,0.6843,35.358281,122867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,9340,2009,32662.890306,10.393995,61170.0,11.021412,854.479941,6.750493,0.217639,0.249040,0.263833,0.269488,0.4995,0.5005,0.6303,0.3697,34.947237,166009
824,9340,2019,45086.834266,10.716346,65792.0,11.094254,1069.550097,6.974993,0.195472,0.232364,0.271028,0.301135,0.5016,0.4984,0.6040,0.3960,36.745534,173839
273,9360,1964,3951.891756,8.281950,15964.0,9.678091,90.895095,4.509706,0.495169,0.301454,0.116320,0.087057,0.5376,0.4624,0.9169,0.0831,27.631902,46235
549,9360,2009,27541.226605,10.223439,86825.0,11.371650,775.000000,6.652863,0.281308,0.259894,0.251303,0.207495,0.5023,0.4977,0.7394,0.2606,36.089917,193995


MSA Level Weighted Mean

In [ ]:
# Calculate the employment-weighted average of logwage for each year
meanlogwage = msa64_19.groupby('year').apply(lambda x: (x['logwage'] * x['emp']).sum() / x['emp'].sum())
meanlogrent = msa64_19.groupby('year').apply(lambda x: (x['logrent'] * x['emp']).sum() / x['emp'].sum())
meanwage = msa64_19.groupby('year').apply(lambda x: (x['wage'] * x['emp']).sum() / x['emp'].sum())


# Reset the index to convert the result into a DataFrame
meanlogwage = meanlogwage.reset_index()
meanlogrent = meanlogrent.reset_index()
meanwage = meanwage.reset_index()

# Rename the columns to have meaningful names
meanlogwage.columns = ['year', 'meanlogwage']
meanlogrent.columns = ['year', 'meanlogrent']
meanwage.columns = ['year', 'meanwage']

# Merge the result back into the original DataFrame
msa64_19 = msa64_19.merge(meanlogwage, on='year', how = 'inner')
msa64_19 = msa64_19.merge(meanlogrent, on='year', how = 'inner')
msa64_19 = msa64_19.merge(meanwage, on='year', how = 'inner')

msa64_19 = msa64_19[['msa', 'year', 'wage', 'meanwage', 'logwage', 'meanlogwage', 'emp', 'logemp', 'rent', 'logrent', 'meanlogrent' , 'drop', 'hs', 'somecoll',
       'collmore', 'male', 'female', 'white', 'nonwhite', 'age', 'total']]

In [ ]:
#Create Housing Cost 
msa64_19['HP'] = msa64_19['rent']*12

In [ ]:
msa64_19.to_csv('../output/msa64_19.csv')
msa64_19.sample(5)

,msa,year,wage,meanwage,logwage,meanlogwage,emp,logemp,rent,logrent,...,hs,somecoll,collmore,male,female,white,nonwhite,age,total,HP
259,8800,1964,3993.212735,5213.814229,8.292351,8.550901,69542.0,11.149686,76.553369,4.337988,...,0.244697,0.105653,0.104280,0.4919,0.5081,0.8386,0.1614,31.180431,150091,918.640427
737,6020,2019,40670.578247,57256.461621,10.613260,10.932744,106010.0,11.571289,698.930950,6.549552,...,0.360197,0.226243,0.321712,0.4871,0.5129,0.9434,0.0566,42.081997,144604,8387.171399
556,280,2019,40777.333658,57256.461621,10.615882,10.932744,106828.0,11.578975,741.000000,6.608001,...,0.443140,0.161417,0.312791,0.4894,0.5106,0.9330,0.0670,42.090032,122495,8892.000000
376,3040,2009,29159.573066,44364.517287,10.280539,10.681799,60712.0,11.013897,564.000000,6.335054,...,0.306487,0.274100,0.321753,0.4943,0.5057,0.8657,0.1343,38.591627,80931,6768.000000
792,8000,2019,45278.327531,57256.461621,10.720584,10.932744,488906.0,13.099926,959.481857,6.866393,...,0.293944,0.183664,0.389292,0.4816,0.5184,0.7498,0.2502,39.903038,698537,11513.782289


In [ ]:
# Assuming df is your DataFrame
pivot = msa64_19.pivot(index='msa', columns='year')

# Flatten the columns by concatenating the levels
pivot.columns = [f"{col[1]}{col[0]}" for col in pivot.columns]

# Reset the index
pivot.reset_index(inplace=True)

# Display the pivoted DataFrame
pivot
pivot.to_csv('../output/pivot.csv')

pivot


,msa,1964wage,2009wage,2019wage,1964meanwage,2009meanwage,2019meanwage,1964logwage,2009logwage,2019logwage,...,2019nonwhite,1964age,2009age,2019age,1964total,2009total,2019total,1964HP,2009HP,2019HP
0,40,4060.505393,29304.201504,38691.262868,5213.814229,44364.517287,57256.461621,8.309063,10.285486,10.563369,...,0.2814,28.023180,35.752957,35.905636,101078.0,130463.0,137521.0,1020.141393,8760.000000,11052.000000
1,120,3713.002915,30555.280130,39549.513354,5213.814229,44364.517287,57256.461621,8.219596,10.327293,10.585309,...,0.7312,25.610463,35.358281,37.459526,81884.0,122867.0,118431.0,966.417281,8000.668319,9461.985419
2,160,5271.284413,39279.277753,51364.691340,5213.814229,44364.517287,57256.461621,8.570029,10.578452,10.846706,...,0.2475,33.303779,38.733936,40.127015,737359.0,918216.0,930060.0,1165.033245,10254.902039,12452.087612
3,200,4884.956515,36050.104377,43946.300716,5213.814229,44364.517287,57256.461621,8.493916,10.492665,10.690724,...,0.3574,26.100653,36.669033,39.143978,315485.0,859226.0,900509.0,1189.403677,9087.329176,10901.948015
4,220,3538.448647,31241.093831,40839.971994,5213.814229,44364.517287,57256.461621,8.171444,10.349490,10.617417,...,0.3871,29.041158,37.036582,38.330383,111351.0,131254.0,130376.0,868.591160,8244.000000,9900.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,4459.251450,33534.905779,42715.112616,5213.814229,44364.517287,57256.461621,8.402736,10.420342,10.662308,...,0.3324,30.445318,34.247319,35.200280,145112.0,239836.0,250649.0,1023.167776,8316.000000,10416.000000
272,9280,4504.762739,36538.681594,44837.711576,5213.814229,44364.517287,57256.461621,8.412891,10.506127,10.710805,...,0.1592,31.787518,38.549658,40.279206,238336.0,431967.0,447628.0,1067.228847,9384.000000,11664.000000
273,9320,5516.502360,31400.863840,38168.179126,5213.814229,44364.517287,57256.461621,8.615499,10.354591,10.549757,...,0.1991,30.858369,40.731398,42.330374,616010.0,559732.0,530110.0,1190.775500,7164.515551,8141.156831
274,9340,4969.140525,32662.890306,45086.834266,5213.814229,44364.517287,57256.461621,8.511002,10.393995,10.716346,...,0.3960,29.168034,34.947237,36.745534,67239.0,166009.0,173839.0,1242.340117,10253.759294,12834.601167


**Metropolitan Statistical Area 2021**

In [ ]:
#Load files to work with
cbp_acs_21 = pd.read_csv('../input/MIRROR_task1_output/cbp_acs_21.csv')
crosswalk = pd.read_csv('../input/MIRROR_task4_output/crosswalk.csv')
#rent23 = pd.read_csv('../input/MIRROR_task1_output/rent23.csv')
cbp_acs_21 = cbp_acs_21.drop(columns = ['Unnamed: 0'])
cbp_acs_21['fips'] = cbp_acs_21['fips'].astype(str).str.zfill(5)
#rent23['fips'] = rent23['fips'].astype(str).str.zfill(5)
crosswalk = crosswalk.sort_values('fips', ascending = True)
crosswalk['fips'] = crosswalk['fips'].astype(str).str.zfill(5)
crosswalk['msa'] = crosswalk['msa'].astype(str).str.zfill(4)

# Merge the county employment data with the crosswalk on the 'fips' column
msa_2021 = cbp_acs_21.merge(crosswalk, on='fips', how='inner')
msa_2021 =msa_2021.dropna(subset=['msa'])
msa_2021 = msa_2021.drop_duplicates(subset=['fips', 'msa'])

# Print the count of duplicate 'fips' values
duplicates_count = msa_2021['fips'].duplicated().sum()
print(f"Number of duplicate 'fips' values: {duplicates_count}")
# Create a boolean mask for duplicated values in the 'fips' column
duplicates_mask = msa_2021['fips'].duplicated(keep=False)
# Use the mask to filter the DataFrame and see the duplicated values
## 10 counties are part of multiple MSA
duplicated_values = msa_2021[duplicates_mask]
duplicates = pd.DataFrame(duplicated_values)

#Create Wage
msa_2021['wage'] = msa_2021['ap'] / msa_2021['emp'] *1000
msa_2021['year'] = msa_2021['year'].astype(int)

msa_2021.to_csv('../output/msa2021.csv')

msa_2021


Number of duplicate 'fips' values: 10


,fips,year,ap,emp,total,male,female,white,non-white,ms-less,hs-drop,hs-above,coll-above,mean-age,rent,msa,wage
0,01003,2021,5197077,132278,239294,115696,123598,198355.0,52998.0,2792.0,11552.0,80314.0,78238.0,42.147864,1096,5160,39289.050333
1,01015,2021,2639238,70223,115972,55581,60391,82072.0,40328.0,2914.0,8632.0,44398.0,22932.0,39.832554,718,0450,37583.669168
2,01051,2021,1114593,29843,89304,44004,45300,66105.0,26759.0,1971.0,7009.0,34361.0,19142.0,39.510895,882,5240,37348.557451
3,01055,2021,2041032,56465,103162,49734,53428,79421.0,29345.0,3435.0,6673.0,41111.0,20804.0,40.611015,717,2880,36146.852032
4,01069,2021,3709119,88319,107458,51800,55658,69907.0,42840.0,2177.0,6156.0,42778.0,23562.0,40.163994,807,2180,41996.840997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,55131,2021,5110712,104911,137175,68944,68231,124849.0,19940.0,1213.0,3237.0,48802.0,44678.0,41.721123,968,5082,48714.739160
1247,55133,2021,28806260,479408,408756,202487,206269,355894.0,75988.0,2920.0,7366.0,108788.0,172073.0,41.779222,1150,5082,60087.149151
1248,55139,2021,9591888,179575,171623,87961,83662,148904.0,31061.0,1356.0,4657.0,59935.0,50740.0,39.497920,807,0460,53414.383962
1249,56021,2021,3453467,69410,100863,50682,50181,82584.0,25603.0,1851.0,3126.0,32015.0,32250.0,39.264319,945,1580,49754.603083


In [ ]:

# Group the data by 'msa' and keep 'year' as is
agg2021 = msa_2021.groupby('msa', as_index=False)['year'].max()

# Calculate the sum of 'total' and 'emp' columns
total_emp_sum = msa_2021.groupby('msa')[['total', 'emp']].sum().reset_index()
agg2021 = agg2021.merge(total_emp_sum, on='msa')

# Calculate the employment-weighted average for other columns
columns_to_average = ['ms-less', 'hs-drop', 'hs-above', 'coll-above', 'rent', 'male', 'female', 'white', 'non-white', 'mean-age', 'wage']
for column in columns_to_average:
    weighted_avg = (msa_2021[column] * msa_2021['emp']).groupby(msa_2021['msa']).sum() / msa_2021.groupby(msa_2021['msa'])['emp'].sum()
    agg2021[column] = weighted_avg.reset_index()[0]

agg2021


,msa,year,total,emp,ms-less,hs-drop,hs-above,coll-above,rent,male,female,white,non-white,mean-age,wage
0,0040,2021,143326,113511,4250.000000,6929.000000,46898.000000,30009.000000,981.000000,70366.000000,72960.000000,96836.000000,71321.000000,36.193929,41255.640423
1,0120,2021,84844,71596,2663.000000,7647.000000,26470.000000,17968.000000,809.000000,39209.000000,45635.000000,19991.000000,68427.000000,38.362813,42531.775518
2,0160,2021,869423,686340,4355.686957,7496.673060,68393.694925,96661.102818,1110.010543,124566.734225,129985.594657,191138.672097,81879.325502,40.482077,57258.645861
3,0200,2021,902952,614395,14985.745642,23359.583835,184400.949705,202856.051174,963.823382,299393.606667,307902.783976,286200.103251,505111.013814,39.797661,47712.987573
4,0220,2021,128654,92604,3841.000000,6304.000000,51711.000000,24303.000000,859.000000,61578.000000,67076.000000,77162.000000,57749.000000,38.847922,44259.416440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,9260,2021,256035,139664,22505.000000,16334.000000,76553.000000,41051.000000,880.000000,128564.000000,127471.000000,114337.000000,213389.000000,35.349470,45550.027208
269,9280,2021,458696,329582,7876.000000,17964.000000,177234.000000,117550.000000,1037.000000,228557.000000,230139.000000,373822.000000,114234.000000,40.499189,48269.304756
270,9320,2021,529407,326761,3306.477181,8661.517895,82490.875208,47557.703101,666.043607,98100.469456,100480.210674,158388.031580,53338.227619,42.357002,40435.963900
271,9340,2021,182484,69659,7092.758854,5744.834953,31069.958799,17366.279634,1146.169727,47704.989018,45979.115678,42406.265436,69217.658192,37.105498,49227.766692


In [ ]:
#Gender Ratio
agg2021['Male'] = agg2021['male'] / (agg2021['male'] + agg2021['female'])
agg2021['Female'] = agg2021['female'] / (agg2021['male'] + agg2021['female'])
agg2021['Male'] = agg2021['Male'].round(4)
agg2021['Female'] = agg2021['Female'].round(4)

#Race Ratio
agg2021['White'] = agg2021['white'] / (agg2021['white'] + agg2021['non-white'])
agg2021['Non-white'] = agg2021['non-white'] / (agg2021['white'] + agg2021['non-white'])
agg2021['White'] = agg2021['White'].round(4)
agg2021['Non-white'] = agg2021['Non-white'].round(4)

#Education Ratio
agg2021['Ms-less'] = agg2021['ms-less'] / agg2021[['ms-less', 'hs-drop', 'hs-above', 'coll-above']].sum(axis=1)
agg2021['Hs-drop'] = agg2021['hs-drop'] / agg2021[['ms-less', 'hs-drop', 'hs-above', 'coll-above']].sum(axis=1)
agg2021['Hs-above'] = agg2021['hs-above'] / agg2021[['ms-less', 'hs-drop', 'hs-above', 'coll-above']].sum(axis=1)
agg2021['Coll-above'] = agg2021['coll-above'] / agg2021[['ms-less', 'hs-drop', 'hs-above', 'coll-above']].sum(axis=1)

agg2021 = agg2021[['msa', 'year', 'wage', 'total', 'emp', 'Ms-less', 'Hs-drop', 'Hs-above', 'Coll-above', 'rent', 'Male', 'Female', 'White', 'Non-white', 'mean-age']]
agg2021
agg2021.to_csv('../output/agg2021.csv')

Append 64, 09, 21 datasets

In [ ]:
import numpy as np
import pandas as pd

agg2021 = pd.read_csv('../input/MIRROR_task4_output/agg2021.csv')
agg2009 = pd.read_csv('../input/MIRROR_task4_output/agg2009.csv')
agg1964 = pd.read_csv('../input/MIRROR_task4_output/agg1964.csv')

final = pd.concat([agg1964, agg2009, agg2021], ignore_index=True)
final = final.sort_values(by=['msa', 'year'], ascending=[True, True])
final = final.drop(columns = 'Unnamed: 0')
final = final.rename(columns = {'Ms-less' : 'ms-less', 'Hs-drop' : 'hs-drop', 'Hs-above':'hs-above', 'Coll-above': 'coll-above',
                                 'Male':'male', 'Female':'female', 'White':'white', 'Non-white':'non-white', 'mean-age':'age'})

final['logwage'] = np.log(final['wage'])
final['logrent'] = np.log(final['rent'])

final = final[['msa', 'year', 'wage', 'logwage', 'total', 'emp', 'ms-less', 'hs-drop', 'hs-above',
       'coll-above', 'rent', 'male', 'female', 'white', 'non-white', 'age','logrent']]

final

,msa,year,wage,logwage,total,emp,ms-less,hs-drop,hs-above,coll-above,rent,male,female,white,non-white,age,logrent
0,40,1964,4060.505393,8.309063,101078,45153.0,0.218807,0.213792,0.444846,0.122554,85.011783,50447.0000,0.499090,0.9530,0.0470,28.023180,4.442790
274,40,2009,29304.201504,10.285486,130463,104891.0,0.039701,0.108088,0.542792,0.309420,730.000000,0.4860,0.514000,0.8129,0.1871,35.752957,6.593045
550,40,2021,41255.640423,10.627543,143326,113511.0,0.048248,0.078662,0.532412,0.340678,981.000000,0.4910,0.509000,0.5759,0.4241,36.193929,6.888572
1,120,1964,3713.002915,8.219596,81884,34300.0,0.297222,0.215533,0.392491,0.094755,80.534773,37781.9499,0.508210,0.6560,0.3440,25.610463,4.388689
275,120,2009,30555.280130,10.327293,122867,89155.0,0.034151,0.164922,0.553987,0.246940,666.722360,0.4632,0.536800,0.3157,0.6843,35.358281,6.502374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,9340,2009,32662.890306,10.393995,166009,61170.0,0.077708,0.139931,0.512873,0.269488,854.479941,0.4995,0.500500,0.6303,0.3697,34.947237,6.750493
821,9340,2021,49227.766692,10.804213,182484,69659.0,0.115755,0.093757,0.507067,0.283421,1146.169727,0.5092,0.490800,0.3799,0.6201,37.105498,7.044181
273,9360,1964,3951.891756,8.281950,46235,15964.0,0.315371,0.179798,0.417774,0.087057,90.895095,21381.0000,0.462442,0.9169,0.0831,27.631902,4.509706
549,9360,2009,27541.226605,10.223439,193995,86825.0,0.113069,0.168239,0.511197,0.207495,775.000000,0.5023,0.497700,0.7394,0.2606,36.089917,6.652863


MSA Level Weighted Mean

In [ ]:
# Calculate the employment-weighted average of logwage for each year
mean_logwage = final.groupby('year').apply(lambda x: (x['logwage'] * x['emp']).sum() / x['emp'].sum())

# Reset the index to convert the result into a DataFrame
mean_logwage = mean_logwage.reset_index()

# Rename the columns to have meaningful names
mean_logwage.columns = ['year', 'mean_logwage']

# Merge the result back into the original DataFrame
final = final.merge(mean_logwage, on='year', how = 'inner')

final.to_csv('../output/final.csv')
final

,msa,year,wage,logwage,total,emp,ms-less,hs-drop,hs-above,coll-above,rent,male,female,white,non-white,age,logrent,mean_logwage
0,40,1964,4060.505393,8.309063,101078,45153.0,0.218807,0.213792,0.444846,0.122554,85.011783,50447.0000,0.499090,0.9530,0.0470,28.023180,4.442790,8.550901
1,120,1964,3713.002915,8.219596,81884,34300.0,0.297222,0.215533,0.392491,0.094755,80.534773,37781.9499,0.508210,0.6560,0.3440,25.610463,4.388689,8.550901
2,160,1964,5271.284413,8.570029,737359,377659.0,0.236249,0.188341,0.441268,0.134142,97.086104,100462.2837,0.519403,0.9674,0.0326,33.303779,4.575598,8.550901
3,200,1964,4884.956515,8.493916,315485,139243.0,0.185235,0.150095,0.496208,0.168461,99.116973,126509.3745,0.502047,0.9659,0.0341,26.100653,4.596301,8.550901
4,220,1964,3538.448647,8.171444,111351,35801.0,0.368398,0.182830,0.358692,0.090080,72.382597,57001.0000,0.511904,0.6946,0.3054,29.041158,4.281966,8.550901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,9260,2021,45550.027208,10.726567,256035,139664.0,0.143854,0.104409,0.489335,0.262402,880.000000,0.5021,0.497900,0.3489,0.6511,35.349470,6.779922,11.021408
819,9280,2021,48269.304756,10.784551,458696,329582.0,0.024565,0.056028,0.552778,0.366629,1037.000000,0.4983,0.501700,0.7659,0.2341,40.499189,6.944087,11.021408
820,9320,2021,40435.963900,10.607475,529407,326761.0,0.023282,0.060989,0.580854,0.334874,666.043607,0.4940,0.506000,0.7481,0.2519,42.357002,6.501355,11.021408
821,9340,2021,49227.766692,10.804213,182484,69659.0,0.115755,0.093757,0.507067,0.283421,1146.169727,0.5092,0.490800,0.3799,0.6201,37.105498,7.044181,11.021408


In [ ]:
# Assuming df is your DataFrame
pivot = final.pivot(index='msa', columns='year')

# Flatten the columns by concatenating the levels
pivot.columns = [f"{col[1]}{col[0]}" for col in pivot.columns]

# Reset the index
pivot.reset_index(inplace=True)

# Display the pivoted DataFrame
pivot
pivot.to_csv('../output/pivot.csv')

pivot


,msa,1964wage,2009wage,2021wage,1964logwage,2009logwage,2021logwage,1964total,2009total,2021total,...,2021non-white,1964age,2009age,2021age,1964logrent,2009logrent,2021logrent,1964mean_logwage,2009mean_logwage,2021mean_logwage
0,40,4060.505393,29304.201504,41255.640423,8.309063,10.285486,10.627543,101078.0,130463.0,143326.0,...,0.4241,28.023180,35.752957,36.193929,4.442790,6.593045,6.888572,8.550901,10.681799,11.021408
1,120,3713.002915,30555.280130,42531.775518,8.219596,10.327293,10.658007,81884.0,122867.0,84844.0,...,0.7739,25.610463,35.358281,38.362813,4.388689,6.502374,6.695799,8.550901,10.681799,11.021408
2,160,5271.284413,39279.277753,57258.645861,8.570029,10.578452,10.955334,737359.0,918216.0,869423.0,...,0.2999,33.303779,38.733936,40.482077,4.575598,6.750604,7.012125,8.550901,10.681799,11.021408
3,200,4884.956515,36050.104377,47712.987573,8.493916,10.492665,10.772959,315485.0,859226.0,902952.0,...,0.6383,26.100653,36.669033,39.797661,4.596301,6.629730,6.870908,8.550901,10.681799,11.021408
4,220,3538.448647,31241.093831,44259.416440,8.171444,10.349490,10.697823,111351.0,131254.0,128654.0,...,0.4281,29.041158,37.036582,38.847922,4.281966,6.532334,6.755769,8.550901,10.681799,11.021408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,9260,4459.251450,33534.905779,45550.027208,8.402736,10.420342,10.726567,145112.0,239836.0,256035.0,...,0.6511,30.445318,34.247319,35.349470,4.445752,6.541030,6.779922,8.550901,10.681799,11.021408
272,9280,4504.762739,36538.681594,48269.304756,8.412891,10.506127,10.784551,238336.0,431967.0,458696.0,...,0.2341,31.787518,38.549658,40.499189,4.487914,6.661855,6.944087,8.550901,10.681799,11.021408
273,9320,5516.502360,31400.863840,40435.963900,8.615499,10.354591,10.607475,616010.0,559732.0,529407.0,...,0.2519,30.858369,40.731398,42.357002,4.597453,6.391989,6.501355,8.550901,10.681799,11.021408
274,9340,4969.140525,32662.890306,49227.766692,8.511002,10.393995,10.804213,67239.0,166009.0,182484.0,...,0.6201,29.168034,34.947237,37.105498,4.639845,6.750493,7.044181,8.550901,10.681799,11.021408
